In [48]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

In [49]:
def standard_format_sql_download(sql_file_name, geo_level, estimates_version):
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=estimates;'
                    'Trusted_Connection=yes;')

    with open(rf'sql_queries\{sql_file_name}.sql', 'r') as sql_file:
        sql_query = sql_file.read()

    sql_query = sql_query.format(geo_level=geo_level, estimates_version=estimates_version)
    df =  pd.read_sql_query(sql_query, conn)
    df = df.rename(columns={'geo_level':geo_level})
    return df

In [51]:
def standard_format_pivot(df, geo_level):
    output = pd.pivot(df, values='value', index=[geo_level, 'yr_id'], columns=['breakdown_value'])
    output.columns.name = ''
    output = output.reset_index()
    return output

In [52]:
def export_to_j_drive(df, geo_level, estimates_version):
    df.to_excel(rf'J:\DataScience\DataQuality\QAQC\estimates_automation\aggregated_data\{geo_level}_est_{estimates_version}_ind_QA.xlsx', index=False)

In [53]:
def standard_format_output_creation(sql_file_name, geo_level, estimates_version):
    sql_output = standard_format_sql_download(sql_file_name=sql_file_name, geo_level=geo_level, estimates_version=estimates_version)
    
    pivoted_output = standard_format_pivot(df=sql_output, geo_level=geo_level)

    export_to_j_drive(df=pivoted_output, geo_level=geo_level, estimates_version=estimates_version)

    return pivoted_output

# Specific Manipulations

In [112]:
def age_manipulations(df, geo_level, estimates_version):
    '''This function puts the columns in the correct order'''
    return df[[geo_level, 'yr_id', 'Under 5', '5 to 9', '10 to 14', '15 to 17', '18 and 19', '20 to 24', '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', '50 to 54', '55 to 59', '60 and 61', '62 to 64', '65 to 69', '70 to 74', '75 to 79', '80 to 84', '85 and Older']]

In [113]:
def households_manipulations(df, geo_level, estimates_version):
    df['Total Households'] = df.loc[:, ~df.columns.isin([geo_level, 'yr_id'])].sum(axis=1)
    return df

In [128]:
def housing_manipulations(df, geo_level, estimates_version):
    df_to_add = standard_format_sql_download(sql_file_name='housing_p2', geo_level=geo_level, estimates_version=estimates_version)

    final = df.merge(df_to_add, how='left', on=[geo_level, 'yr_id'])

    return final 

In [143]:
def income_manipulations(df, geo_level, estimates_version):
    '''This function puts the columns in the correct order'''
    return df[[geo_level, 'yr_id', 'Less than $15,000', '$15,000 to $29,999', '$30,000 to $44,999', '$45,000 to $59,999', '$60,000 to $74,999', '$75,000 to $99,999', '$100,000 to $124,999', '$125,000 to $149,999', '$150,000 to $199,999', '$200,000 or more']]

In [152]:
def population_manipulations(df, geo_level, estimates_version):
    df['Total Population'] = df.loc[:, ~df.columns.isin([geo_level, 'yr_id'])].sum(axis=1)
    df['Total GQ Population'] = df.loc[:, ~df.columns.isin([geo_level, 'yr_id', 'Household Population', 'Total Population'])].sum(axis=1)
    return df

# Information Table

In [167]:
table_information = {
    'age': {
            'manipulation_function': age_manipulations
    },
    'children':{
            'manipulation_function': None
    },
    'ethnicity':{
            'manipulation_function': None
    },
    'households':{
            'manipulation_function': households_manipulations
    },
    'housing':{
            'manipulation_function': housing_manipulations
    },
    'income':{
            'manipulation_function': income_manipulations
    },
    'population':{
            'manipulation_function': population_manipulations
    },
    'sex':{
            'manipulation_function': None
    },
    'workers':{
            'manipulation_function': None
    }
}

# Output Function

In [165]:
pivot_needed = ['age', 'ethnicity', 'households', 'housing', 'income', 'population', 'sex']

In [169]:
def standard_format_output_creation(table_key_name, geo_level, estimates_version):
    output = standard_format_sql_download(sql_file_name=table_key_name, geo_level=geo_level, estimates_version=estimates_version)

    if table_key_name in pivot_needed:
        output = standard_format_pivot(df=output, geo_level=geo_level)

    if table_information[table_key_name]['manipulation_function'] != None:
        output = table_information[table_key_name]['manipulation_function'](df=output, geo_level=geo_level, estimates_version=estimates_version)

    export_to_j_drive(df=output, geo_level=geo_level, estimates_version=estimates_version)

    return output

# Create Output

In [170]:
for table_key_name in table_information.keys():
    for geo_level in ['mgra', 'census_tract', 'luz', 'cpa', 'jurisdiction', 'sra', 'region']:
        standard_format_output_creation(table_key_name=table_key_name, geo_level=geo_level, estimates_version='2022_01')
        print(f"{table_key_name}-{geo_level} is complete.")